## Data Prep Merge

In [40]:
import pandas as pd

# merge df_umsatz_warengruppe, df_kiel_events, df_kiwo, df_Schiffahrt, df_wetter, df_wochentage
# df_umsatz_warengruppe is the main dataframe, add the others via left merge
# all dataframes are in the folder _Variables_cleaned

df_umsatz_warengruppe = pd.read_csv('../_Variables_cleaned/df_umsatz_warengruppe.csv')
df_kiel_events = pd.read_csv('../_Variables_cleaned/df_kiel_events.csv')
df_kiwo = pd.read_csv('../_Variables_cleaned/df_kiwo.csv')
# df_Schifffahrt = pd.read_csv('../_Variables_cleaned/df_Schifffahrt.csv')
df_wetter = pd.read_csv('../_Variables_cleaned/df_wetter.csv')
df_wochentage = pd.read_csv('../_Variables_cleaned/df_wochentage.csv')

df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_kiel_events, on='Datum', how='outer')
df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_kiwo, on='Datum', how='outer')
# df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_Schifffahrt, on='Datum', how='left')
df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_wetter, on='Datum', how='outer')
df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_wochentage, on='Datum', how='outer')

# remove column Wochentag_Mo 
df_umsatz_warengruppe = df_umsatz_warengruppe.drop(columns=['Wochentag_Mo'])

# remove column Brot
df_umsatz_warengruppe = df_umsatz_warengruppe.drop(columns=['Saisonbrot'])

# save dataframe as df_prepared in _data_prepared
df_umsatz_warengruppe.to_csv('../_data_prepared/df_dataset_complete.csv', index=False)

In [41]:
# check whether all rows are unique in df_umsatz_warengruppe. If not, drop duplicates. 
df_umsatz_warengruppe = pd.read_csv('../_data_prepared/df_dataset_complete.csv')

# check for duplicates
# df_umsatz_warengruppe_duplicates = df_umsatz_warengruppe[df_umsatz_warengruppe.duplicated()]

# drop duplicates
df_umsatz_warengruppe = df_umsatz_warengruppe.drop_duplicates()

# save dataframe in _data_prepared
df_umsatz_warengruppe.to_csv('../_data_prepared/df_dataset_complete.csv', index=False)

## Filter dataset_complete: Remove data before 01.07.2013 and after 30.07.2019

In [42]:
# filter out all rows before 01.07.2013 and after 30.07.2019
import pandas as pd

# read in df_dataset_complete
df_umsatz_warengruppe = pd.read_csv('../_data_prepared/df_dataset_complete.csv')

# filter out all rows before 01.07.2013 and after 30.07.2019
df_umsatz_warengruppe = df_umsatz_warengruppe[(df_umsatz_warengruppe['Datum'] >= '2013-07-01') & (df_umsatz_warengruppe['Datum'] <= '2019-07-30')]

# save df
df_umsatz_warengruppe.to_csv('../_data_prepared/df_dataset_complete.csv', index=False)


# Sorge dafür, dass es für jeden tag je Warengruppe gibt

In [43]:
# Ich brauche je Datum 6 Zeilen. Prüfe daher für jedes Datum, ob es 6 Zeilen gibt. Wenn nicht, füge 5 Zeilen hinzu.
df_umsatz_warengruppe = pd.read_csv('../_data_prepared/df_dataset_complete.csv')

# check for each date if there are 6 rows. If not, add 5 rows
dates = df_umsatz_warengruppe['Datum'].unique()

rows_to_add = []

for date in dates:
    if df_umsatz_warengruppe[df_umsatz_warengruppe['Datum'] == date].shape[0] != 6:
        for i in range(1, 7):
            if df_umsatz_warengruppe[(df_umsatz_warengruppe['Datum'] == date) & (df_umsatz_warengruppe['Warengruppe'] == i)].shape[0] == 0:
                rows_to_add.append({'Datum': date, 'Warengruppe': i})

df_umsatz_warengruppe = pd.concat([df_umsatz_warengruppe, pd.DataFrame(rows_to_add)], ignore_index=True)

# sort the df by Datum and Warengruppe
df_umsatz_warengruppe = df_umsatz_warengruppe.sort_values(by=['Datum', 'Warengruppe'])

# save the df
# df_umsatz_warengruppe_.to_csv('../_data_prepared/df_dataset_complete.csv', index=False)


# Umsatz = 0, wenn NA und gesetzlicher Feiertag

In [44]:
# Umsatz = 0, wenn Umsatz = NA und national_holiday = 1, aber nur bis 31.07.2018
df_umsatz_warengruppe.loc[(df_umsatz_warengruppe['Umsatz'].isnull()) & (df_umsatz_warengruppe['national_holiday'] == 1) & (df_umsatz_warengruppe['Datum'] <= '2018-07-31'), 'Umsatz'] = 0

# save the df
df_umsatz_warengruppe.to_csv('../_data_prepared/df_dataset_complete.csv', index=False)

# Anpassung der Daten
## Umsatz = 0, wenn NA und andere Umsätze für andere Warengruppe an gleichem Tag 
## Umsatz (Saisonbrot) = 0, wenn nicht verkauft 

In [45]:
# set all NA values in df_umsatz_warengruppe['Umsatz'] to 0. But only when there are other non-NA values on the same day
# get all dates where Umsatz is NA
dates = df_umsatz_warengruppe[df_umsatz_warengruppe['Umsatz'].isnull()]['Datum'].unique()

# iterate over all dates
for date in dates:
    # get all rows for this date
    rows = df_umsatz_warengruppe[df_umsatz_warengruppe['Datum'] == date]
    # check if there are other non-NA values for this date
    if rows['Umsatz'].notnull().sum() > 0:
        # set all NA values to 0
        df_umsatz_warengruppe.loc[df_umsatz_warengruppe['Datum'] == date, 'Umsatz'] = df_umsatz_warengruppe.loc[df_umsatz_warengruppe['Datum'] == date, 'Umsatz'].fillna(0)


# set all NA values in df_umsatz_warengruppe['Umsatz'] to 0 when column Warengruppe = 6
df_umsatz_warengruppe.loc[df_umsatz_warengruppe['Warengruppe'] == 6, 'Umsatz'] = df_umsatz_warengruppe.loc[df_umsatz_warengruppe['Warengruppe'] == 6, 'Umsatz'].fillna(0)

# save the df
df_umsatz_warengruppe.to_csv('../_data_prepared/df_dataset_complete.csv', index=False)

## Add dummies for Monat

In [46]:
# add dummies for column Monat
df_umsatz_warengruppe = pd.get_dummies(df_umsatz_warengruppe, columns=['Monat'], drop_first=True)
# Convert month columns to integer
df_umsatz_warengruppe = df_umsatz_warengruppe.astype({col: int for col in df_umsatz_warengruppe.columns if 'Monat_' in col})

# Rename month columns to remove ".0"
df_umsatz_warengruppe.columns = [col.replace('.0', '') if 'Monat_' in col else col for col in df_umsatz_warengruppe.columns]

# save dataframe as df_prepared in _data_prepared
df_umsatz_warengruppe.to_csv('../_data_prepared/df_dataset_complete.csv', index=False)


## Behandlung von Duplicates in all Dataframes
### Find all duplicates in df_dataset_complete

In [47]:
# find all exact duplicates rows in df_dataset_complete
import pandas as pd

df = pd.read_csv('../_data_prepared/df_dataset_complete.csv')

# Find duplicates
duplicates = df[df.duplicated(keep=False)]

# remove duplicates, keep only the first occurence, remove all rows that are completely empty
df_dropDuplicates = df.drop_duplicates()

# save df
df_dropDuplicates.to_csv('../_data_prepared/df_dataset_complete.csv', index=False)


## Divide Dataset

In [48]:
import pandas as pd

# read data as df (df_merged_umsatz_kiwo01_wetter_wochentage.csv) in 1_DatasetCharacteristics
df = pd.read_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_complete.csv')

# Den Datensatz df teilen in Trainingsdatensatz and Validierungsdatensatz
# df_training: alle Daten vom 01.07.2013 bis 31.07.2017
# df_validation: alle Daten vom 01.08.2017 bis 31.07.2018
# df_test: alle Daten vom 01.08.2018 bis 30.07.2019

df_training = df[(df['Datum'] >= '2013-07-01') & (df['Datum'] < '2017-08-01')]
df_validation = df[(df['Datum'] >= '2017-08-01') & (df['Datum'] < '2018-08-01')]
df_test = df[(df['Datum'] >= '2018-08-01') & (df['Datum'] <= '2019-07-30')]


# save df_training and df_test as csv files
df_training.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_training.csv', index=False)
df_validation.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_validation.csv', index=False)
df_test.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_test.csv', index=False)


# Imuptation (Umsatz) with knn

In [ ]:
# Imputiere Umsatz (knn) für die Warengruppen 1-5 (aber nur bis 31.07.2018)
# knn-imputation of column Umsatz for Warengruppe 1
from sklearn.impute import KNNImputer

# Convert Datum to datetime format in df_test and df_validation
df_training['Datum'] = pd.to_datetime(df_training['Datum'])
df_validation['Datum'] = pd.to_datetime(df_validation['Datum'])

# Convert Datum to numerical values for imputation
df_training['Datum_numeric'] = df_training['Datum'].map(pd.Timestamp.toordinal)
df_validation['Datum_numeric'] = df_validation['Datum'].map(pd.Timestamp.toordinal)

# Select columns for imputation
columns_for_imputation = [
    'Datum_numeric', 'Umsatz', 'Warengruppe', 'national_holiday', 'christmas_market', 
    'KielerWoche', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode', 
    'Wochentag_Di', 'Wochentag_Mi', 'Wochentag_Do', 'Wochentag_Fr', 'Wochentag_Sa', 
    'Wochentag_So', 'Monat_2', 'Monat_3', 'Monat_4', 'Monat_5', 'Monat_6', 'Monat_7', 
    'Monat_8', 'Monat_9', 'Monat_10', 'Monat_11', 'Monat_12'
]

# impute missing values with knn for df_training
imputer = KNNImputer(n_neighbors=5)
df_training_imputed = df_training.copy()
df_training_imputed['Umsatz'] = imputer.fit_transform(df_training[columns_for_imputation])[:, columns_for_imputation.index('Umsatz')]

# impute missing values with knn for df_validation
df_validation_imputed = df_validation.copy()
df_validation_imputed['Umsatz'] = imputer.fit_transform(df_validation[columns_for_imputation])[:, columns_for_imputation.index('Umsatz')]

# Drop the Datum_numeric column
df_training_imputed = df_training_imputed.drop(columns=['Datum_numeric'])
df_validation_imputed = df_validation_imputed.drop(columns=['Datum_numeric'])

# Change column Datum: from datetime to string (format: yyyy-mm-dd)
df_training_imputed['Datum'] = df_training_imputed['Datum'].dt.strftime('%Y-%m-%d')
df_validation_imputed['Datum'] = df_validation_imputed['Datum'].dt.strftime('%Y-%m-%d')



/tmp/ipykernel_56572/1129770628.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['Datum'] = pd.to_datetime(df_training['Datum'])
/tmp/ipykernel_56572/1129770628.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_validation['Datum'] = pd.to_datetime(df_validation['Datum'])
/tmp/ipykernel_56572/1129770628.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
# es soll für jedes Datum 6 Zeilen geben. Zeige mir jedes Datum, die mehr oder weniger als 6 Zeilen hat
# check for each date if there are 6 rows. If not, show them to me
dates = df_training_imputed['Datum'].unique()

for date in dates:
    if df_training_imputed[df_training_imputed['Datum'] == date].shape[0] != 6:
        print(date, df_training_imputed[df_training_imputed['Datum'] == date].shape[0])

# If there are more than 6 rows for a date, drop the row with NA in column Warengruppe
for date in dates:
    if df_training_imputed[df_training_imputed['Datum'] == date].shape[0] > 6:
        print(df_training_imputed[(df_training_imputed['Datum'] == date) & (df_training_imputed['Warengruppe'].isnull())])
        df_training_imputed = df_training_imputed.drop(df_training_imputed[(df_training_imputed['Datum'] == date) & (df_training_imputed['Warengruppe'].isnull())].index)



# Repeat the same for df_validation_imputed
dates_validation = df_validation_imputed['Datum'].unique()

for date in dates_validation:
    if df_validation_imputed[df_validation_imputed['Datum'] == date].shape[0] != 6:
        print(date, df_validation_imputed[df_validation_imputed['Datum'] == date].shape[0])

# If there are more than 6 rows for a date, drop the row with NA in column Warengruppe
for date in dates_validation:
    if df_validation_imputed[df_validation_imputed['Datum'] == date].shape[0] > 6:
        print(df_validation_imputed[(df_validation_imputed['Datum'] == date) & (df_validation_imputed['Warengruppe'].isnull())])
        df_validation_imputed = df_validation_imputed.drop(df_validation_imputed[(df_validation_imputed['Datum'] == date) & (df_validation_imputed['Warengruppe'].isnull())].index)



2013-12-25 7
2013-12-26 7
2013-12-28 7
2014-01-01 7
2014-04-18 7
2014-05-01 7
2014-05-03 7
2014-05-04 7
2014-08-17 7
2014-12-25 7
2014-12-26 7
2015-01-01 7
2015-04-03 7
2015-05-01 7
2015-12-25 7
2015-12-26 7
2016-01-01 7
2016-03-24 7
2016-03-25 7
2016-07-07 7
2016-07-18 7
2016-08-14 7
2016-08-15 7
2016-12-17 7
2016-12-25 7
2016-12-26 7
2017-01-01 7
2017-04-14 7
2017-05-01 7
2017-07-04 7
           Datum  Umsatz  Warengruppe  Brot  Broetchen  Croissant  Konditorei  \
1068  2013-12-25     0.0          NaN   NaN        NaN        NaN         NaN   

      Kuchen  national_holiday  christmas_market  ...  Monat_3  Monat_4  \
1068     NaN               1.0               0.0  ...        0        0   

      Monat_5  Monat_6  Monat_7  Monat_8  Monat_9  Monat_10  Monat_11  \
1068        0        0        0        0        0         0         0   

      Monat_12  
1068         1  

[1 rows x 47 columns]
           Datum  Umsatz  Warengruppe  Brot  Broetchen  Croissant  Konditorei  \
1075  2013-

# Imputation temp_bins (knn)

In [51]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

# Funktion zur k-NN Imputation
def knn_imputation(df, columns):
    imputer = KNNImputer(n_neighbors=5)
    df[columns] = imputer.fit_transform(df[columns])
    return df

# Funktion zur Anpassung der temp_bins
def adjust_temp_bins(df, columns):
    for i in range(len(df)):
        max_col = df.loc[df.index[i], columns].idxmax()
        for column in columns:
            df.loc[df.index[i], column] = 1 if column == max_col else 0
    return df

# k-NN Imputation für 'temp_bins_kalt', 'temp_bins_mild', 'temp_bins_warm', 'temp_bins_heiß'
columns_to_impute = ['temp_bins_kalt', 'temp_bins_mild', 'temp_bins_warm', 'temp_bins_heiß']
df_training_imputed = knn_imputation(df_training_imputed, columns_to_impute)
df_validation_imputed = knn_imputation(df_validation_imputed, columns_to_impute)

# Anpassung der temp_bins
df_training_imputed = adjust_temp_bins(df_training_imputed, columns_to_impute)
df_validation_imputed = adjust_temp_bins(df_validation_imputed, columns_to_impute)

# Überprüfen der Ergebnisse
print("Anzahl der fehlenden Werte nach der Imputation (Training):")    
print(df_training_imputed[columns_to_impute].isnull().sum())

print("Anzahl der fehlenden Werte nach der Imputation (Validation):")    
print(df_validation_imputed[columns_to_impute].isnull().sum())

# Initialisierung der letzten gültigen Werte
# last_valid_values = {column: 0 for column in data.keys()}


Anzahl der fehlenden Werte nach der Imputation (Training):
temp_bins_kalt    0
temp_bins_mild    0
temp_bins_warm    0
temp_bins_heiß    0
dtype: int64
Anzahl der fehlenden Werte nach der Imputation (Validation):
temp_bins_kalt    0
temp_bins_mild    0
temp_bins_warm    0
temp_bins_heiß    0
dtype: int64


# Add dummies for newly added values

In [52]:
# for all NAs in columns Brot, Broetchen, Croissant, Konditorei, Kuchen, Saisonbrot: add dummies based on the following mapping (1: 'Brot', 2: 'Broetchen', 3: 'Croissant', 4: 'Konditorei', 5: 'Kuchen', 6: 'Saisonbrot'). The dummie columns already exist in the dataframe.

# if Warengruppe = 1, set Brot = 1 and all other columns to 0
# if Warengruppe = 2, set Broetchen = 1 and all other columns to 0
# if Warengruppe = 3, set Croissant = 1 and all other columns to 0
# if Warengruppe = 4, set Konditorei = 1 and all other columns to 0
# if Warengruppe = 5, set Kuchen = 1 and all other columns to 0
# if Warengruppe = 6, set Saisonbrot = 1 and all other columns to 0

for df in [df_training_imputed, df_validation_imputed]:
    df['Brot'] = np.where(df['Warengruppe'] == 1, 1, 0)
    df['Broetchen'] = np.where(df['Warengruppe'] == 2, 1, 0)
    df['Croissant'] = np.where(df['Warengruppe'] == 3, 1, 0)
    df['Konditorei'] = np.where(df['Warengruppe'] == 4, 1, 0)
    df['Kuchen'] = np.where(df['Warengruppe'] == 5, 1, 0)
    df['Saisonbrot'] = np.where(df['Warengruppe'] == 6, 1, 0)


In [53]:
# für fehelende Werte in den folgenden Spalten (national_holiday, christmas_market, KielerWoche, Wochentag_Di, Wochentag_Mi, Wochentag_Do, Wochentag_Fr, Wochentag_Sa, Wochentag_So), übernimm den Wert aus Warengruppe 1 am gleichen Tag
columns_to_impute = ['national_holiday', 'christmas_market', 'KielerWoche', 'Wochentag_Di', 'Wochentag_Mi', 'Wochentag_Do', 'Wochentag_Fr', 'Wochentag_Sa', 'Wochentag_So']
for column in columns_to_impute:
    df_training_imputed[column] = df_training_imputed.groupby('Datum')[column].transform('first')
    df_validation_imputed[column] = df_validation_imputed.groupby('Datum')[column].transform('first')


In [54]:
# remove column Saisonbrot
df_training_imputed = df_training_imputed.drop(columns=['Saisonbrot'])
df_validation_imputed = df_validation_imputed.drop(columns=['Saisonbrot']) 

In [55]:
# rename df_training_imputed and df_validation_imputed to df_training and df_validation
df_training = df_training_imputed
df_validation = df_validation_imputed

# save the dataframes
df_training_imputed.to_csv('../_data_prepared/df_training.csv', index=False)
df_validation_imputed.to_csv('../_data_prepared/df_validation.csv', index=False)

## Prepare df_test for prediction
### Adding Wochentag

In [9]:
# add dummy value to existing columns Wochentag_Mo, Wochentag_Di, Wochentag_Mi, Wochentag_Do, Wochentag_Fr, Wochentag_Sa, Wochentag_So to df_test depending on the weekday of the date in column Datum (JJJ-MM-TT HH:MM:SS)

import pandas as pd

# Load the dataset
df_test = pd.read_csv('../_data_prepared/df_test.csv')

# Convert Datum to datetime
df_test['Datum'] = pd.to_datetime(df_test['Datum'])

# Create dummy columns for weekdays
df_test['Wochentag_Mo'] = (df_test['Datum'].dt.dayofweek == 0).astype(int)
df_test['Wochentag_Di'] = (df_test['Datum'].dt.dayofweek == 1).astype(int)
df_test['Wochentag_Mi'] = (df_test['Datum'].dt.dayofweek == 2).astype(int)
df_test['Wochentag_Do'] = (df_test['Datum'].dt.dayofweek == 3).astype(int)
df_test['Wochentag_Fr'] = (df_test['Datum'].dt.dayofweek == 4).astype(int)
df_test['Wochentag_Sa'] = (df_test['Datum'].dt.dayofweek == 5).astype(int)
df_test['Wochentag_So'] = (df_test['Datum'].dt.dayofweek == 6).astype(int)

# Save the updated dataset
df_test.to_csv('../_data_prepared/df_test.csv', index=False)



## Add all value of Warengruppe to the df (6 rows for each date)

In [10]:
# duplicate rows in df_test so that every day is represented 6 times, with value of 1 to 6 in column Warengruppe for each value of Datum
import pandas as pd

# Load the dataset
df_test = pd.read_csv('../_data_prepared/df_test.csv')

# Create a new DataFrame with 6 duplicates for each row
df_test_expanded = pd.concat([df_test] * 6, ignore_index=True)

# Sort the DataFrame by Datum and Warengruppe
df_test_expanded = df_test_expanded.sort_values(['Datum', 'Warengruppe'])

# Add a new column 'Warengruppe' with values from 1 to 6
df_test_expanded['Warengruppe'] = df_test_expanded.groupby('Datum').cumcount() + 1

# add dummy encoding from Warengruppe to df_test_expanded (1 = Brot, 2 = Brötchen, 3 = Croissant, 4 = Konditorei, 5 = Kuchen, 6 = Saisonbrot), the columns are already named correctly
df_test_expanded['Brot'] = (df_test_expanded['Warengruppe'] == 1).astype(int)
df_test_expanded['Broetchen'] = (df_test_expanded['Warengruppe'] == 2).astype(int)
df_test_expanded['Croissant'] = (df_test_expanded['Warengruppe'] == 3).astype(int)
df_test_expanded['Konditorei'] = (df_test_expanded['Warengruppe'] == 4).astype(int)
df_test_expanded['Kuchen'] = (df_test_expanded['Warengruppe'] == 5).astype(int)
df_test_expanded['Saisonbrot'] = (df_test_expanded['Warengruppe'] == 6).astype(int)

# Save the updated dataset
df_test_expanded.to_csv('../_data_prepared/df_test.csv', index=False)


## create the column id as a combination of date and Warengruppe (as needed for kaggle upload)

In [11]:
# insert column id df_test as first column from Datum and Warengruppe, format: "YYMMDDWarengruppe", Warengruppe in df_test is a number from 1 to 6 (the dataframe is already loaded)
import pandas as pd

# Load the dataset
df_test = pd.read_csv('../_data_prepared/df_test.csv')

# Convert Datum to datetime
df_test['Datum'] = pd.to_datetime(df_test['Datum'])

# Create the ID column
df_test['id'] = df_test['Datum'].dt.strftime('%y%m%d') + df_test['Warengruppe'].astype(str)

# Reorder the columns
df_test = df_test[['id'] + [col for col in df_test.columns if col != 'id']]

# Save the updated dataset
df_test.to_csv('../_data_prepared/df_test.csv', index=False)



## Merge created df with sample submission (df template)

In [12]:
# merge with sample submission 
import pandas as pd

# Load the datasets
df = pd.read_csv('../2_BaselineModel/sample_submission.csv')

# remove Umsatz from df
df = df.drop(columns=['Umsatz'])

df_test = pd.read_csv('../_data_prepared/df_test.csv')

# merge sample_submission with df_test based on column id
df_test = pd.merge(df, df_test, on='id', how='left')

# Save the updated dataset
df_test.to_csv('../_data_prepared/df_test.csv', index=False)
